## Importing Modules that will be used

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import *
from scipy import stats
import seaborn as sns

## Setting number format to display in this notebook

In [2]:
pd.options.display.float_format = '{:,.2f}'.format

## Creating functions that will be used in this notebook

In [3]:
def string_tdate(x):
    return datetime.fromisoformat(x)

## Importing dataframes from internet
##### url: https://ourworldindata.org/covid-vaccinations

In [4]:
df_vaccine = pd.read_csv('data/covid-vaccine-doses-by-manufacturer.csv')
df_covid = pd.read_csv('owid-covid-data.csv')

# 1 - Cleaning dataset

### Applying date functions to change format to datetime

In [5]:
df_vaccine.Day = df_vaccine.Day.apply(string_tdate)
df_covid.date = df_covid.date.apply(string_tdate)

In [6]:
df_covid.new_deaths = df_covid.new_deaths.apply(lambda x: x if x >= 0 else 0)
df_covid.new_deaths = df_covid.new_deaths.fillna(value = 0)

### Cleaning columns from dataset that won't be used

In [7]:
df_covid_2 = df_covid.loc[:,['continent','location','date','total_cases','new_cases','total_deaths',\
                             'new_deaths','new_tests','total_tests','total_vaccinations',\
                             'people_vaccinated','people_fully_vaccinated','population','hospital_beds_per_thousand']]

# 2 - Creating new columns for analysis

### Creating a column to get month and year from the dates in the df

In [8]:
df_covid_2['month_year'] = pd.to_datetime(df_covid_2["date"].dt.strftime('%Y-%m'))
df_vaccine['month_year'] = pd.to_datetime(df_vaccine["Day"].dt.strftime('%Y-%m'))
df_covid_2['new_deaths'] = df_covid_2['new_deaths'].fillna(value = 0)
df_covid_2['total_deaths'] = df_covid_2['total_deaths'].fillna(value = 0)
df_covid_2['people_vaccinated'] = df_covid_2['people_vaccinated'].fillna(value = 0)
df_covid_2['people_fully_vaccinated'] = df_covid_2['people_fully_vaccinated'].fillna(value = 0)

### Getting start date for vaccination in each country

In [9]:
cond_1 = df_covid_2['people_vaccinated'] == 0
init_vac = df_covid_2.loc[~cond_1,['location','date']].groupby(by='location').min().reset_index()
init_vac.rename(columns={'date':'date_init_vac'}, inplace = True)
df_covid_3 = df_covid_2.merge(init_vac, how = 'left', on = 'location')
# Sorting data frame by country and date
df_covid_3.sort_values(by=['location', 'date'], ascending = [True, True]).reset_index(drop = True);
df_covid_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97585 entries, 0 to 97584
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   continent                   92997 non-null  object        
 1   location                    97585 non-null  object        
 2   date                        97585 non-null  datetime64[ns]
 3   total_cases                 94172 non-null  float64       
 4   new_cases                   94169 non-null  float64       
 5   total_deaths                97585 non-null  float64       
 6   new_deaths                  97585 non-null  float64       
 7   new_tests                   43837 non-null  float64       
 8   total_tests                 43532 non-null  float64       
 9   total_vaccinations          15498 non-null  float64       
 10  people_vaccinated           97585 non-null  float64       
 11  people_fully_vaccinated     97585 non-null  float64   

In [10]:
cond_1 = df_covid_3['people_fully_vaccinated'] == 0
init_vac = df_covid_3.loc[~cond_1,['location','date']].groupby(by='location').min().reset_index()
init_vac.rename(columns={'date':'date_init_vac_full'}, inplace = True)
df_covid_31 = df_covid_3.merge(init_vac, how = 'left', on = 'location')
# Sorting data frame by country and date
df_covid_31.sort_values(by=['location', 'date'], ascending = [True, True]).reset_index(drop = True);
df_covid_31.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97585 entries, 0 to 97584
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   continent                   92997 non-null  object        
 1   location                    97585 non-null  object        
 2   date                        97585 non-null  datetime64[ns]
 3   total_cases                 94172 non-null  float64       
 4   new_cases                   94169 non-null  float64       
 5   total_deaths                97585 non-null  float64       
 6   new_deaths                  97585 non-null  float64       
 7   new_tests                   43837 non-null  float64       
 8   total_tests                 43532 non-null  float64       
 9   total_vaccinations          15498 non-null  float64       
 10  people_vaccinated           97585 non-null  float64       
 11  people_fully_vaccinated     97585 non-null  float64   

In [11]:
df_covid_3 = df_covid_31

In [12]:
for row in range(len(df_covid_3)):
    if df_covid_3.iloc[row,2] < df_covid_3.iloc[row,16]:
        pass
    elif df_covid_3.iloc[row,11] == 0:
        row_int = row
        while df_covid_3.iloc[row_int,11] == 0:
            row_int = row_int-1
        df_covid_3.iloc[row,11] = df_covid_3.iloc[row_int,11]

In [13]:
for row in range(len(df_covid_3)):
    if df_covid_3.iloc[row,2] < df_covid_3.iloc[row,15]:
        pass
    elif df_covid_3.iloc[row,10] == 0:
        row_int = row
        while df_covid_3.iloc[row_int,10] == 0:
            row_int = row_int-1
        df_covid_3.iloc[row,10] = df_covid_3.iloc[row_int,10]

### Creating a recovered column:
#### Recovered Cases (D0) = Total Cases (D0 - AVG Days to Recover) - Total Deaths (D0)

In [14]:
result = []
days_to_recover = 22
countries_list = list(df_covid_3.location.unique()) 

for country in countries_list:
    df_filtered = df_covid_3.loc[df_covid_3.location==country,:].copy()
    
    for row in range(len(df_filtered)):
        if row < 22:
            recovered = 0
            result.append(recovered)
        else:
            try:
                recovered = int(df_filtered.iloc[row-days_to_recover,3] - df_filtered.iloc[row,6])
                if recovered < 0:
                    0
                else:
                    recovered
            except:
                recovered = 0
            result.append(recovered)

df_concat = pd.DataFrame({'total_recovered':result})    
df_covid_4 = pd.concat([df_covid_3,df_concat], axis =1 )

### Creating column to identifie when Country's population achieves over XPTO % of immunization.
#### % Immunization will be calculated in 4 different ways

In [15]:
#df_covid_4['pop_immunized'] = df_covid_4['total_recovered'] + df_covid_4['people_vaccinated']
#df_covid_4['pop_immunized'] = df_covid_4['total_recovered'] + df_covid_4['people_fully_vaccinated']
#df_covid_4['pop_immunized'] = df_covid_4['people_vaccinated']
df_covid_4['pop_immunized'] = df_covid_4['people_fully_vaccinated']

df_covid_4['immunized_percentage'] = df_covid_4['pop_immunized']/df_covid_4['population']
df_covid_4['immunized_percentage'] = df_covid_4['immunized_percentage'].apply(lambda x: 1 if x > 1.0 else x)

In [16]:
df_covid_4['death_pop'] = df_covid_4['total_deaths']/df_covid_4['population']
df_covid_4['cases_pop'] = df_covid_4['total_cases']/df_covid_4['population']

In [17]:
### Creating two dataframes: before and after immunization
percentage = 0.35
cond_1 = df_covid_4.immunized_percentage >= percentage
cond_3 = df_covid_4.new_deaths > 0
im_countries = list(df_covid_4.loc[cond_1,'location'].unique())
p_value_ind = []
p_value_levene = []

for country in im_countries:
    cond_2 = df_covid_4.location == country
    
    df_after = df_covid_4.loc[cond_1&cond_2,:]
    df_before = df_covid_4.loc[~cond_1&cond_2&cond_3,:]
    #df_before = df_covid_4.loc[~cond_1&cond_2,:]
    
    x_ind = stats.ttest_ind(df_after['new_deaths'], df_before['new_deaths'].dropna())
    y = stats.levene(df_after['new_deaths'], df_before['new_deaths'].dropna())
    
    p_value_ind.append(x_ind[1])
    p_value_levene.append(y[1])
    
df_test = pd.DataFrame({'country': im_countries , 'p_value_levene': p_value_levene, 'p_value_ind': p_value_ind })

/usr/local/Cellar/jupyterlab/3.0.0/libexec/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/Cellar/jupyterlab/3.0.0/libexec/lib/python3.9/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [18]:
len(im_countries)

42

#### Checking how many countries have the same var before and after. Like a filter to our tests.

In [19]:
cond_I = df_test.p_value_levene > 0.05
cond_II = df_test.p_value_ind <= 0.05
len(df_test.dropna().loc[cond_I, :].sort_values(by='p_value_ind' , ascending = False ))

6

#### Checking how many countries have the same var before and after and REJECTED our null hipothesis

In [20]:
cond_I = df_test.p_value_levene > 0.05
cond_II = df_test.p_value_ind <= 0.05
len(df_test.dropna().loc[cond_I&cond_II, :].sort_values(by='p_value_ind' , ascending = False ))

6

#### Checking how many countries have the same var before and after and CONFIRMED our null hipothesis

In [21]:
cond_I = df_test.p_value_levene > 0.05
cond_II = df_test.p_value_ind > 0.05
len(df_test.dropna().loc[cond_I&cond_II, :].sort_values(by='p_value_ind' , ascending = False ))

0

In [22]:
df_test.dropna().sort_values(by='p_value_ind' , ascending = False )

,country,p_value_levene,p_value_ind
28,Serbia,0.03,0.10
34,United Arab Emirates,0.00,0.03
37,Uruguay,0.20,0.02
22,Mongolia,0.01,0.02
30,Singapore,0.07,0.02
29,Seychelles,0.68,0.00
25,Qatar,0.12,0.00
6,Chile,0.77,0.00
35,United Kingdom,0.00,0.00
13,Iceland,0.13,0.00


In [23]:
### Creating two dataframes: before and after immunization
percentage = 0.35
cond_1 = df_covid_4.immunized_percentage >= percentage
cond_3 = df_covid_4.new_deaths > 0
country = 'United Kingdom'
cond_2 = df_covid_4.location == country
df_after = df_covid_4.loc[cond_1&cond_2,:]
df_before = df_covid_4.loc[~cond_1&cond_2&cond_3,:]
print(df_before.new_deaths.mean(),df_after.new_deaths.mean())

288.94130925507903 9.678571428571429


In [24]:
### Creating two dataframes: before and after immunization
percentage = 0.35
cond_1 = df_covid_4.immunized_percentage >= percentage
cond_2 = df_covid_4.new_deaths > 0
df_after = df_covid_4.loc[cond_1&cond_2,:]
df_before = df_covid_4.loc[~cond_1&cond_2,:]

In [25]:
df_after.date.min()

Timestamp('2020-03-26 00:00:00')

In [27]:
df_covid_4.to_excel('covid_output.xlsx')